
<div hidden>

$\gdef\dd{\mathrm{d}}$

</div>

<div hidden>

$\gdef\abs#1{\left\vert#1\right\vert}$

</div>

<div hidden>

$\gdef\ve#1{\bm{#1}}$
</div>

<div hidden>

$\gdef\mat#1{\mathbf{#1}}$
</div>

<div hidden>
Authors: Felipe Uribe and Amal Alghamdi
</div>

# Two forward models

## <font color='blue'> Contents of this notebook: </font>
  * [Learning objectives](#r-learning-objectives)
  * [Forward model: 1D Poisson](#r-forward-model-1d-poisson)
  * [Forward model: 1D convolution](#r-forward-model-1d-convolution)


## <font color='blue'> Learning objectives: </font> <a name="r-learning-objectives"></a>
  * Create (or load pre-existing) linear and non-linear forward models in CUQIpy and use them
  * Create and plot input for the forward models and compute and plot the corresponding output
  * Mathematically define two forward models: 1D Poisson and 1D Convolution and use them in CUQIpy
  

In [ ]:
from cuqi.testproblem import Poisson1D, Deconvolution1D, Heat1D
from cuqi.distribution import Gaussian
from cuqi.array import CUQIarray
import numpy as np
import matplotlib.pyplot as plt

## <font color='blue'> Forward model: 1D Poisson </font> <a name="r-forward-model-1d-poisson"></a>

Consider a heat conductive rod of length $L = \pi$ with a varying conductivity (the conductivity of the rod changes from point to point). We fix the temperature at the end-points of the rod and apply a heat source distributed along the length of the rod. We wait until the rod reaches an equilibrium temperature distribution. The equilibrium temperature of the rod is modelled using the second order steady-state PDE as

$$
\left\{
\begin{aligned}
& \dfrac{\dd}{\dd \xi}\left(u(\xi) \dfrac{\dd y(\xi)}{\dd \xi}\right) = -f(\xi), \quad & \xi\in (0,L) \\
& y(0) = y(L) = 0.
\end{aligned}
\right.
$$
Here, $y$ represents the temperature distribution along the rod, $u(\xi) $ is the unknown conductivity of the rod and $f(\xi)$ is a deterministic heat source given by

$$
\begin{aligned}
	f(\xi) = 10\exp( -\frac{ (\xi - L/2)^2} {0.02} ).
\end{aligned}
$$

To ensure that the conductivity of the rod is non-negative, we parameterize $u$ by a random variable $x$ as follows:
 
$$
 \begin{aligned}
 u( \cdot  ) = \exp( x( \cdot  ) )
 \end{aligned}
$$
where $x$ is not necessarily positive.

Let us load the forward model that maps the random variable $x$ to the temperature distribution $y$ in CUQIpy. We will use the following parameters:
* `dim` : number of discretization points for the rod
* `L` : length of the rod
* `f` : a function that represents the heat source

In [ ]:
dim = 128
L = np.pi
f = lambda xi: 10*np.exp(-(xi-L/2)**2 / 0.02)

Then we can load the 1D Poisson forward model as follows:

In [ ]:

A = Poisson1D(dim=dim, endpoint=L, source=f).model

We print the forward model to see its details.

In [ ]:
A

We can look at the domain and range geometries of the forward model.

In [ ]:
print(A.domain_geometry)
print(A.range_geometry)

These geometries are of type `Continuous1D` which represents a 1D continuous signal/field defined on a grid. We can view the grid:

In [ ]:
print(A.domain_geometry.grid)

Additionally, the properties `domain_dim` and `range_dim` of the forward model represent the dimension of the input and output of the forward model, respectively.

In [ ]:
print(A.domain_dim)
print(A.range_dim)

Let us create an array representing a constant conductivity over the grid nodes 

In [ ]:
some_x_array = 20*np.ones(A.domain_dim)

We can wrap the array in a `CUQIarray` object which is the main data structure in CUQIpy for variables (e.g. arrays and fields)

In [ ]:
some_x = CUQIarray(some_x_array, geometry=A.domain_geometry)

Note that we pass `geometry=A.domain_geometry` to equip the `CUQIarray` object with the same geometry as the domain of the forward model, which is has information about what the values of the array represent (e.g. function values on a grid for this case).

We can plot the conductivity array using the `plot` method of the `CUQIarray` object

In [ ]:
some_x.plot()

We can also evaluate the forward model at the conductivity array and plot the solution:

In [ ]:
some_y = A(some_x)
some_y.plot()

#### <font color='magenta'> Exercise: </font>
  * Is this true for you: **"I can, mathematically, write a definition of a 1D Poisson forward model that maps the conductivity field to the temperature distribution field"**? If not, what questions do you have?
  * Trying a different conductivity profile: 
      * Create another constant conductivity profile `another_x` of value 20 as a `CUQIarray` object.
      * Evaluate the forward model at the new conductivity profile and store the result in a variable `another_y`. Plot the solution.
      * In one plot, compare the solutions `some_y` and `another_y` using the `plot` method of the `CUQIarray` object. What do you observe? and why?
  * Experimenting with setting up the `map` in `Poisson1D`:
      * execute `help(Poisson1D)`
      * Note the `map` parameter of `Poisson1D`. We can use this parameter to transform the conductivity field before solving the PDE. Create the forward model again, name it `A_map`, with setting up `map` to be `lambda x: np.exp(x)` to ensure that the conductivity is always positive.
      * Inspect the domain and range geometries of the new forward model `A_map` what is different this time, and why?
      * create a CUQIarray object `x_with_map` with a constant conductivity profile of value 0 and evaluate the forward model at `x_with_map` and store it in variable `y_with_map`. Make sure to use the right geometry for `x_with_map`.
      * Plot `x_with_map` using `x_with_map.plot()`, What do you observe?
      * Plot the solution `y_with_map` and `some_y` in the same plot. What do you observe? and why?
  * Experimenting with setting up the `field_type` in `Poisson1D`:
      * Note the `field_type` parameter of `Poisson1D`. We can use this parameter to change the parameterization of the conductivity field. Create the forward model again, name it `A_step`, with setting both the `map` as in the previous exercise and the `field_type` to be `"Step"` to generate a step parameterization of the conductivity field.
      * What is the domain and range geometries of the new forward model `A_step`?
      * Create `x_step` to be `x_step = CUQIarray(np.arange(A_step.domain_dim)*0.1, geometry=A_step.domain_geometry)`.
      * Plot `x_step` using `x_step.plot()`.
      * What does the dimension `A_step.domain_dim` represent? and is it equal to the size of the domain geometry grid?
      Evaluate the forward model at `x_step` and store it in variable `y_step`. Plot the solution.
    * Is this true for you: **"I can load pre-existing forward models in CUQIpy and evaluate them on some input and visualize the input and output"**? If not, what questions do you have?

## <font color='blue'> Forward model: 1D convolution </font> <a name="r-forward-model-1d-convolution"></a>

The mathematical model for convolution of a random signal on a one-dimensional spatial domain $D=[0,1]$, can be written as a stochastic Fredholm integral equation of the first kind:
$$
y(x,\omega) = \int_{0}^{1} k(x,x')X(x',\omega)\,\dd x' \quad\text{with}\quad k(x,x') = \frac{b}{2\exp\left(-b\abs{x-x'}\right)},
$$
where $y(x,\omega)$ denotes the convolved signal random variable and we assume a deterministic convolution kernel $k$ with fixed parameter $b=48$. In practice, a finite-dimensional representation of \eqref{eq:1D_deconv} is employed. After discretizing the signal domain into $N$ components, the convolution model can be expressed as a system of linear algebraic equations $\ve{y}(\omega)=\mat{K}\ve{x}(\omega)$. 

This problem is implemented in the CUQI toolbox as $$\texttt{model}=\texttt{cuqi.model.Deconv\_1D}(N, L, \texttt{kernel}, \texttt{field\_type}, \texttt{KL\_map}),$$
where $N$ is the domain discretization, $L=1$ is the length of the domain, $\texttt{kernel}$ is a function with the convolution kernel implementation $\texttt{field\_type}=\{\texttt{None},\texttt{KL},\texttt{CustomKL},\texttt{Step}\}$ defines the random field generation (specified in the projects), and $\texttt{KL\_map}$ is a function with the transformation of a Gaussian field (in our case is always $x=\exp(u)$). To obtain a forward solution on the domain $D$ given a Gaussian parameter vector, we simply use $\texttt{y~=~model.forward(u)}$